In [1]:
import torch
from datasets import load_dataset,Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    Trainer

)
from peft import LoraConfig, get_peft_model, TaskType

c:\Users\Roshan Acharya\Documents\Roshan\Projects\ArthaBodh.ai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def format_instruction(sample):

    if sample['input']:
        text = f"### Instruction:\n{sample['instruction']}\n\n### Context:\n{sample['input']}\n\n### Response:\n{sample['response']}"
    else:
        text = f"### Instruction:\n{sample['instruction']}\n\n### Response:\n{sample['response']}"
    return {"text": text}

In [3]:
import json
RAW_JSON = "instruction_data.json"
MODEL_NAME = "Qwen/Qwen3-1.7B"

In [4]:
with open(RAW_JSON, "r", encoding="utf-8") as f:
    raw_data = json.load(f)




formatted = [format_instruction(s) for s in raw_data]
dataset = load_dataset("json", data_files={"train": RAW_JSON, "test": "instruction_data_val.json"})


In [5]:
dataset = dataset.map(format_instruction)
dataset

Map: 100%|██████████| 30/30 [00:00<00:00, 1153.83 examples/s]


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'text'],
        num_rows: 415
    })
    test: Dataset({
        features: ['instruction', 'input', 'response', 'text'],
        num_rows: 30
    })
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 311/311 [00:05<00:00, 59.93it/s, Materializing param=model.norm.weight]                               
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [8]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=1024)

tokenized_dataset = dataset.map(tokenize, batched=True)

Map: 100%|██████████| 30/30 [00:00<00:00, 425.07 examples/s]


In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'text', 'input_ids', 'attention_mask'],
        num_rows: 415
    })
    test: Dataset({
        features: ['instruction', 'input', 'response', 'text', 'input_ids', 'attention_mask'],
        num_rows: 30
    })
})

In [10]:
columns_to_keep = ["input_ids", "attention_mask"]  # for causal LM
tokenized_dataset = tokenized_dataset.remove_columns(["instruction", "input", "response", "text"])


In [11]:
def add_labels(batch):
    batch["labels"] = batch["input_ids"].copy()
    return batch

tokenized_dataset = tokenized_dataset.map(add_labels, batched=False)


Map: 100%|██████████| 30/30 [00:00<00:00, 450.83 examples/s]


In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [13]:
model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./qwen-finance-nepali",
    per_device_train_batch_size=1,   # Reduced to 1 for 6GB VRAM safety
    per_device_eval_batch_size=1,    # Reduced to 1 for 6GB VRAM safety
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,               
    
 
    eval_strategy="steps",           
    save_strategy="steps",           
    eval_steps=100,                  # Evaluation interval
    save_steps=100,                  # Save interval
    
    save_total_limit=2,
    load_best_model_at_end=True,
    gradient_accumulation_steps=8,  
    report_to="none"                
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, padding=True, return_tensors="pt")

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator
)


In [ ]:
import torch
print(torch.cuda.is_available())  
print(torch.cuda.device_count())  


True
1


In [18]:
trainer.train()

Step,Training Loss,Validation Loss
100,0.287483,0.302820


TrainOutput(global_step=156, training_loss=1.1711589113259926, metrics={'train_runtime': 7646.0407, 'train_samples_per_second': 0.163, 'train_steps_per_second': 0.02, 'total_flos': 1.31857236099072e+16, 'train_loss': 1.1711589113259926, 'epoch': 3.0})

In [19]:
# Save the LoRA adapters and tokenizer
model.save_pretrained("./qwen_nepali_budget_lora")
tokenizer.save_pretrained("./qwen_nepali_budget_lora")

print("✅ Fine-tuned model saved successfully")


✅ Fine-tuned model saved successfully
